In [1]:
from hdx.hdx_configuration import Configuration 
from hdx.data.dataset import Dataset
import requests
import pandas as pd
import numpy as np
import csv
from zipfile import ZipFile
import os

In [2]:
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

# Population

In [ ]:
prediction_data  = pd.read_csv('../data/train/pop/fr/departements-francais.csv', sep=';')
prediction_data.columns = ['dep_num', 'name', 'region', 'capital', 'area', 'total', 'density']
prediction_data = prediction_data.sort_values('dep_num')
prediction_data = prediction_data[:-5]
prediction_data['region'] = prediction_data['region'].replace({'Ile-de-France':'Île-de-France'})
prediction_data

# Covid

In [ ]:
#import requests
#import pandas as pd
#import datetime
#departements = pd.read_csv("data/pop/fr/departements-francais.csv", sep = ";")
#hospi = []
#url = "https://coronavirusapi-france.now.sh/AllLiveData"
#response = requests.get(url).json()
#counter = 0
#for numero in departements.NUMÉRO:
#    nom = str(departements[departements["NUMÉRO"]==numero]["NOM"].values[0])
#    hospi.append((nom, numero, response["allLiveFranceData"][counter]["nouvellesHospitalisations"]))
#    counter+=1
#
#df = pd.DataFrame(hospi, columns =["depname","depnum","newhospi"])
#print(df)
#df.to_csv("covid_daily_data.csv", index = False)

# CAMS

In [ ]:
#to_do (Niclas ?)

# Mobility

In [ ]:
Configuration.create(hdx_site='prod', user_agent='A_Quick_Example', hdx_read_only=True)
dataset = Dataset.read_from_hdx('movement-range-maps')
resources = dataset.get_resources()
dic = resources[1]
url_mobility = dic['download_url']

file_mobility = "../data/prediction/mvt_range.zip"
download_url(url_mobility, file_mobility)

In [ ]:
with ZipFile(file_mobility, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting mv_range file now...')
    mvt_range = zip.namelist()[-1]
    zip.extract(mvt_range)
    print('Done!')

In [ ]:
with open(mvt_range) as f:
    reader = csv.reader(f, delimiter="\t")
    d = list(reader)

In [ ]:
data_mob = pd.DataFrame(d[1:], columns=d[0])
data_mob = data_mob[data_mob['country']=='FRA']

In [ ]:
data_mob = data_mob[data_mob['ds']==list(data_mob.iloc[[-1]]['ds'])[0]][['ds', 'polygon_name', 'all_day_bing_tiles_visited_relative_change', 'all_day_ratio_single_tile_users']]

In [ ]:
prediction_data['stay_put'] = 0.0
prediction_data['go_out'] = 0.0

In [ ]:
def add_go_out(row):
    region = row['region']
    go_out = data_mob[data_mob['polygon_name']==region]['all_day_bing_tiles_visited_relative_change']
    return float(list(go_out)[0])
     
def add_stay_put(row):   
    region = row['region']
    stay_put = data_mob[data_mob['polygon_name']==region]['all_day_ratio_single_tile_users']
    return float(list(stay_put)[0])

In [ ]:
prediction_data['go_out'] = prediction_data.apply(add_go_out, axis=1)
prediction_data['stay_put'] = prediction_data.apply(add_stay_put, axis=1)
prediction_data

# Vaccins

In [ ]:
url_positive_test = 'https://www.data.gouv.fr/es/datasets/r/59aeab47-c364-462c-9087-ce233b6acbbc'
download_url(url_positive_test, '../data/prediction/live_vaccins.csv')

In [ ]:
live_vacc = pd.read_csv('../data/prediction/live_vaccins.csv')
live_vacc['date_debut_semaine'] = pd.to_datetime(live_vacc['date_debut_semaine'])
date_max = live_vacc['date_debut_semaine'].max()

In [ ]:
vacc_1 = live_vacc[live_vacc['rang_vaccinal']==1]
vacc_2 = live_vacc[live_vacc['rang_vaccinal']==2]

In [ ]:
def live_vacc_1(row):
    dep = row['dep_num']
    vacc_1_reg = vacc_1[vacc_1['code_region']==dep]
    if vacc_1_reg.shape[0]!=0:
        nb_series = vacc_1_reg[vacc_1_reg['date_debut_semaine']==date_max]['nb']
        nb = list(nb_series)[0]
    else : 
        nb=0
    return nb

def live_vacc_2(row):
    dep = row['dep_num']
    vacc_2_reg = vacc_2[vacc_2['code_region']==dep]
    if vacc_2_reg.shape[0]!=0:
        nb_series = vacc_2_reg[vacc_2_reg['date_debut_semaine']==date_max]['nb']
        nb = list(nb_series)[0]
    else : 
        nb=0
    return nb

In [ ]:
prediction_data['vacc_1'] = prediction_data.apply(live_vacc_1, axis=1)
prediction_data['vacc_2'] = prediction_data.apply(live_vacc_2, axis=1)
prediction_data

# Positive tests

In [3]:
url_positive = 'https://www.data.gouv.fr/en/datasets/r/406c6a23-e283-4300-9484-54e78c8ae675'

with requests.Session() as s:
    download = s.get(url_positive)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=';')
    my_list = list(cr)

In [4]:
live_positive = pd.DataFrame(my_list[1:],columns=my_list[0])

live_positive

,dep,jour,P,T,cl_age90,pop
0,01,2020-05-13,0,16,09,83001
1,01,2020-05-13,1,17,19,84665
2,01,2020-05-13,0,34,29,65496
3,01,2020-05-13,1,72,39,85588
4,01,2020-05-13,0,54,49,89678
...,...,...,...,...,...,...
387811,978,2021-04-16,0,29,69,3060.02499200135
387812,978,2021-04-16,0,6,79,1152.93437717932
387813,978,2021-04-16,0,0,89,346.012162334751
387814,978,2021-04-16,0,0,90,70.6551671722845


In [51]:
live_positive['dep'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '2A', '2B', '30', '31', '32',
       '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43',
       '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
       '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65',
       '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76',
       '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87',
       '88', '89', '90', '91', '92', '93', '94', '95', '971', '972',
       '973', '974', '975', '976', '977', '978'], dtype=object)

In [50]:
def replace_dep(row):
    if type(row['dep']) != int:
        row['dep'] = int(row['dep'].replace('2A','201').replace('2B','202'))
    return None

In [49]:
live_positive.apply(replace_dep, axis=1)

3718      None
3719      None
3720      None
3721      None
3722      None
          ... 
357979    None
357980    None
357981    None
357982    None
357983    None
Length: 1056, dtype: object

In [52]:
def P_to_int(row):
    return int(row['P'])
live_positive['P']= live_positive.apply(P_to_int, axis=1)

In [53]:
live_positive= live_positive[live_positive["dep"]<203]
live_positive["jour"]=pd.to_datetime(live_positive["jour"], dayfirst = True)

In [54]:
live_positive = live_positive[live_positive['jour']==live_positive['jour'].max()]

In [55]:
live_positive['P'].unique()

array([   8,   46,   38,   55,   64,   47,   25,   10,    3,    1,  297,
         19,   48,   50,   56,   58,   43,   39,   14,  338,    5,   13,
         16,   17,   22,    9,  117,   15,    7,   18,    2,    0,   84,
          6,   11,   12,   36,   67,   51,   44,   49,    4,  336,   23,
        118,   26,   21,  135,   53,   34,   28,  159,  145,   24,   85,
         99,  212,  294,  302,  273,  249,  162,   88, 1738,   54,   41,
         27,  286,   37,   20,   29,  178,  137,  116,   30,  173,  210,
        120,   70,   62,   35,  246,   33,  267,   40,  228,   45,   60,
         65,   92,   86,   75,   31,  475,   68,   96,   90,  464,   78,
         95,   79,   42,  531,   83,  111,   87,  106,   76,  572,   82,
         74,  465,   81,   63,   52,  339,   97,  115,  578,  109,  156,
         61,  512,  148,  503,  349,   89,   66,  342,  305,  121,   59,
        346,  253,  392,   93,  301,  293,  334,  279,  218,  131,   32,
       1756,  105,  107,  667,  147,  163,  138,  1

In [57]:
live_positive = live_positive.groupby('dep').sum().reset_index()

In [58]:
live_positive

,dep,P
0,1,594
1,2,676
2,3,234
3,4,168
4,5,110
...,...,...
91,93,3414
92,94,2436
93,95,2388
94,201,90
